In [1]:
import numpy as np
import pandas as pd

### Datasets

In [2]:
df = pd.read_csv('C:/Users/Nithin/Downloads/Segmentation_Heuristic/DATA SET FOR RFM-VALUE BASED SEGMENTATION.csv')

### Data Prep

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22602 entries, 0 to 22601
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   cust_id               22602 non-null  int64  
 1   No_of_days_frm_L_pur  22602 non-null  int64  
 2   total_sale            22602 non-null  float64
 3   No_baskets            22602 non-null  int64  
 4   item_cnt              22602 non-null  int64  
 5   no_SKUs               22602 non-null  int64  
 6   no_Categories         22602 non-null  int64  
 7   avg_no_prds_w_ret     22602 non-null  float64
 8   Flag_Multi_cat_p2     22602 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 1.6 MB


In [4]:
df.rename(columns={'No_of_days_frm_L_pur':'recency','total_sale':'monetary_value','No_baskets':'frequency'},inplace=True)

### Value based Segmentation

In [5]:
df['decile'] = pd.qcut(df.monetary_value,10,labels=False)

In [6]:
df.columns

Index(['cust_id', 'recency', 'monetary_value', 'frequency', 'item_cnt',
       'no_SKUs', 'no_Categories', 'avg_no_prds_w_ret', 'Flag_Multi_cat_p2',
       'decile'],
      dtype='object')

In [7]:
cols = ['recency', 'monetary_value', 'frequency', 'item_cnt',
       'no_SKUs', 'no_Categories', 'avg_no_prds_w_ret', 'Flag_Multi_cat_p2']

In [8]:
VBS = df.groupby('decile').apply(lambda x : x[cols].mean())

In [9]:
VBS.T.round(2).to_csv('C:/Users/Nithin/Downloads/Segmentation_Heuristic/VBS.csv')

### RFM Segementation

In [10]:
quantiles = df[['recency','monetary_value','frequency']].quantile(q = [0.25,0.5,0.75])

In [13]:
quantiles = quantiles.to_dict()

In [15]:
#UDF
def R_class (x,p,d):
    if x <= d[p][0.25]:
        return 1
    if x <= d[p][0.50]:
        return 2
    if x <= d[p][0.75]:
        return 3
    else:
        return 4
    
def F_class (x,p,d):
    if x <= d[p][0.25]:
        return 4
    if x <= d[p][0.50]:
        return 3
    if x <= d[p][0.75]:
        return 2
    else:
        return 1 

In [22]:
df['R_quantile'] = df['recency'].apply(R_class,args=('recency',quantiles))
df['F_quantile'] = df['frequency'].apply(F_class,args=('frequency',quantiles))
df['M_quantile'] = df['monetary_value'].apply(F_class,args=('monetary_value',quantiles))

In [28]:
df1 = df.assign(R_seg=np.nan, FM_seg =np.nan)
df1.R_seg[df['R_quantile']==4] = "3.Churn"
df1.R_seg[df['R_quantile']==2] = "2.At Risk"
df1.R_seg[df['R_quantile']==3] = "2.At Risk"
df1.R_seg[df['R_quantile']==1] = "1.Active"

df1.FM_seg[(df['F_quantile']==3) & (df['M_quantile']==1)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==4) & (df['M_quantile']==1)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==2) & (df['M_quantile']==2)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==2) & (df['M_quantile']==1)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==3) & (df['M_quantile']==2)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==4) & (df['M_quantile']==2)] = "1.Silverf"
df1.FM_seg[(df['F_quantile']==1) & (df['M_quantile']==3)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==2) & (df['M_quantile']==3)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==3) & (df['M_quantile']==3)] = "1.Silver"
df1.FM_seg[(df['F_quantile']==4) & (df['M_quantile']==3)] = "1.Silver"
df1.FM_seg[(df['F_quantile']==1) & (df['M_quantile']==4)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==1) & (df['M_quantile']==2)] = "2.Gold"
df1.FM_seg[(df['F_quantile']==2) & (df['M_quantile']==4)] = "1.Silver"
df1.FM_seg[(df['F_quantile']==3) & (df['M_quantile']==4)] = "1.Silver"
df1.FM_seg[(df['F_quantile']==4) & (df['M_quantile']==4)] = "0.Standard"

df1.FM_seg.fillna("4.Premium", inplace=True)

C:\Users\Nithin\AppData\Local\Temp/ipykernel_28572/4005109935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.R_seg[df['R_quantile']==4] = "3.Churn"
C:\Users\Nithin\AppData\Local\Temp/ipykernel_28572/4005109935.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.R_seg[df['R_quantile']==2] = "2.At Risk"
C:\Users\Nithin\AppData\Local\Temp/ipykernel_28572/4005109935.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.R_seg

In [33]:
RFM = df1.groupby(['R_seg','FM_seg']).apply(lambda x : x[cols].mean()).T

In [35]:
RFM.to_csv('C:/Users/Nithin/Downloads/Segmentation_Heuristic/RFM.csv')